In [5]:
from torchkge.utils.datasets import load_fb13, load_fb15k

In [2]:
train, valid, test = load_fb13('data')

In [4]:
test.rel2ix

{'religion': 0,
 'cause_of_death': 1,
 'place_of_death': 2,
 'profession': 3,
 'location': 4,
 'gender': 5,
 'nationality': 6,
 'place_of_birth': 7,
 'institution': 8,
 'children': 9,
 'parents': 10,
 'spouse': 11,
 'ethnicity': 12}

In [6]:
train, valid, test = load_fb15k('data')

In [7]:
test.ent2ix

{'/m/010016': 0,
 '/m/0100mt': 1,
 '/m/0102t4': 2,
 '/m/0104lr': 3,
 '/m/0105y2': 4,
 '/m/0106dv': 5,
 '/m/0108xl': 6,
 '/m/0109vk': 7,
 '/m/010bnr': 8,
 '/m/010bxh': 9,
 '/m/010cw1': 10,
 '/m/010dft': 11,
 '/m/010h9y': 12,
 '/m/010hn': 13,
 '/m/010m55': 14,
 '/m/010nlt': 15,
 '/m/010p3': 16,
 '/m/010r6f': 17,
 '/m/010rvx': 18,
 '/m/010t4v': 19,
 '/m/010tkc': 20,
 '/m/010v8k': 21,
 '/m/010xjr': 22,
 '/m/010y34': 23,
 '/m/010z5n': 24,
 '/m/0113sg': 25,
 '/m/0114m0': 26,
 '/m/0118d3': 27,
 '/m/0118v': 28,
 '/m/011_3s': 29,
 '/m/011_6p': 30,
 '/m/011_vz': 31,
 '/m/011hdn': 32,
 '/m/011hq1': 33,
 '/m/011j5x': 34,
 '/m/011k11': 35,
 '/m/011k1h': 36,
 '/m/011k4g': 37,
 '/m/011k_j': 38,
 '/m/011kn2': 39,
 '/m/011lpr': 40,
 '/m/011lvx': 41,
 '/m/011pcj': 42,
 '/m/011s0': 43,
 '/m/011s9r': 44,
 '/m/011v3': 45,
 '/m/011vx3': 46,
 '/m/011w20': 47,
 '/m/011w4n': 48,
 '/m/011w54': 49,
 '/m/011wdm': 50,
 '/m/011wtv': 51,
 '/m/011x_4': 52,
 '/m/011xg5': 53,
 '/m/011xhx': 54,
 '/m/011xjd': 55,
 '/m/01

In [8]:
!pip install googledrivedownloader

In [10]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='0B52yRXcdpG6MaHA5ZW9CZ21MbVk',
                                    dest_path='data/mid2name.tsv.gz',
                                    unzip=True)

Unzipping...

In [17]:
from pandas import read_csv

mid2name = read_csv('data/mid2name.tsv.gz',sep='\t',compression='gzip', names=['mid','entity'])
mid2ent = {i:e for i,e in zip(mid2name['mid'],mid2name['entity'])}
mid2ent

{'/m/0gw0': 'Anarcism',
 '/m/0gwj': 'Autisim',
 '/m/0gx0': 'Albedo rate',
 '/m/0gxj': 'Abû Zabi',
 '/m/0gy0': 'A.',
 '/m/0gyh': 'Alabama (state)',
 '/m/0gy_': 'Achillies',
 '/m/0gzh': 'President Abraham',
 '/m/0gz_': 'The Stagirite',
 '/m/0g_h': 'AnAmericanInParis',
 '/m/0g_w': 'The Oscars',
 '/m/0h0c': 'Animalia (book)',
 '/m/0h0q': 'Atomic time',
 '/m/0h16': 'Altruistically',
 '/m/0h1p': '李安',
 '/m/0h25': 'Ayb rand',
 '/m/0h2p': 'Allan Connes',
 '/m/0h34': 'Allen Dwan',
 '/m/0h3y': "Ad-Dimuqratiyah ash-Sha'biyah",
 '/m/0h53': 'Ayn Rand Atlas Shrugged',
 '/m/0h5k': 'Historical anthropologist',
 '/m/0h61': 'Archeology',
 '/m/0h6k': 'The Agricultural Sciences',
 '/m/0h70': 'Alchemist',
 '/m/0h7j': 'Automatic dependent surveillance broadcast',
 '/m/0h7x': 'Austrian Republic',
 '/m/0h8d': 'America Samoa',
 '/m/0h9c': 'Astronom',
 '/m/0h9v': 'Theratromyxa',
 '/m/0hb8': 'Bemer-Ross Code',
 '/m/0hcr': 'Animated images',
 '/m/0hd7': 'Phanaeus',
 '/m/0hdr': 'Andre Kirk Agassi',
 '/m/0hf7': 'Mo

In [1]:
from torch import cuda
from torch.optim import Adam

from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
from torchkge.utils.datasets import load_fb15k
from tqdm.autonotebook import tqdm

In [2]:
cuda.is_available()

True

In [4]:
# Load dataset
kg_train, kg_valid, kg_test = load_fb15k()

# Define some hyper-parameters for training
emb_dim = 100
lr = 0.0004
n_epochs = 100
b_size = 32768
margin = 0.5

# Define the model and criterion
model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2', )
criterion = MarginLoss(margin)

# Move everything to CUDA if available
if cuda.is_available():
    cuda.empty_cache()
    model.cuda()
    criterion.cuda()
    
# Define the torch optimizer to be used
optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)

# KG training options
sampler = BernoulliNegativeSampler(kg_train)
dataloader = DataLoader(kg_train, batch_size=b_size, use_cuda='all')

# Training
iterator = tqdm(range(n_epochs), unit='epoch')
for epoch in iterator:
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        h, t, r = batch[0], batch[1], batch[2]
        n_h, n_t = sampler.corrupt_batch(h, t, r)

        optimizer.zero_grad()

        # forward + backward + optimize
        pos, neg = model(h, t, n_h, n_t, r)
        loss = criterion(pos, neg)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    iterator.set_description(
        'Epoch {} | mean loss: {:.5f}'.format(epoch + 1, running_loss / len(dataloader)))
model.normalize_parameters()

  0%|          | 0/100 [00:00<?, ?epoch/s]

In [6]:
ent, rel = model.get_embeddings()

In [8]:
ent.shape

torch.Size([14951, 100])

In [14]:
dataloader.h

483142

In [16]:
kg_train.ent2ix

{'/m/010016': 0,
 '/m/0100mt': 1,
 '/m/0102t4': 2,
 '/m/0104lr': 3,
 '/m/0105y2': 4,
 '/m/0106dv': 5,
 '/m/0108xl': 6,
 '/m/0109vk': 7,
 '/m/010bnr': 8,
 '/m/010bxh': 9,
 '/m/010cw1': 10,
 '/m/010dft': 11,
 '/m/010h9y': 12,
 '/m/010hn': 13,
 '/m/010m55': 14,
 '/m/010nlt': 15,
 '/m/010p3': 16,
 '/m/010r6f': 17,
 '/m/010rvx': 18,
 '/m/010t4v': 19,
 '/m/010tkc': 20,
 '/m/010v8k': 21,
 '/m/010xjr': 22,
 '/m/010y34': 23,
 '/m/010z5n': 24,
 '/m/0113sg': 25,
 '/m/0114m0': 26,
 '/m/0118d3': 27,
 '/m/0118v': 28,
 '/m/011_3s': 29,
 '/m/011_6p': 30,
 '/m/011_vz': 31,
 '/m/011hdn': 32,
 '/m/011hq1': 33,
 '/m/011j5x': 34,
 '/m/011k11': 35,
 '/m/011k1h': 36,
 '/m/011k4g': 37,
 '/m/011k_j': 38,
 '/m/011kn2': 39,
 '/m/011lpr': 40,
 '/m/011lvx': 41,
 '/m/011pcj': 42,
 '/m/011s0': 43,
 '/m/011s9r': 44,
 '/m/011v3': 45,
 '/m/011vx3': 46,
 '/m/011w20': 47,
 '/m/011w4n': 48,
 '/m/011w54': 49,
 '/m/011wdm': 50,
 '/m/011wtv': 51,
 '/m/011x_4': 52,
 '/m/011xg5': 53,
 '/m/011xhx': 54,
 '/m/011xjd': 55,
 '/m/01

In [22]:
kg_train.ent2ix.index(15)

AttributeError: 'dict' object has no attribute 'index'

In [18]:
from collections import defaultdict

In [20]:
ent2idx = defaultdict(int, kg_train.ent2ix)

In [21]:
ent2idx

defaultdict(int,
            {'/m/010016': 0,
             '/m/0100mt': 1,
             '/m/0102t4': 2,
             '/m/0104lr': 3,
             '/m/0105y2': 4,
             '/m/0106dv': 5,
             '/m/0108xl': 6,
             '/m/0109vk': 7,
             '/m/010bnr': 8,
             '/m/010bxh': 9,
             '/m/010cw1': 10,
             '/m/010dft': 11,
             '/m/010h9y': 12,
             '/m/010hn': 13,
             '/m/010m55': 14,
             '/m/010nlt': 15,
             '/m/010p3': 16,
             '/m/010r6f': 17,
             '/m/010rvx': 18,
             '/m/010t4v': 19,
             '/m/010tkc': 20,
             '/m/010v8k': 21,
             '/m/010xjr': 22,
             '/m/010y34': 23,
             '/m/010z5n': 24,
             '/m/0113sg': 25,
             '/m/0114m0': 26,
             '/m/0118d3': 27,
             '/m/0118v': 28,
             '/m/011_3s': 29,
             '/m/011_6p': 30,
             '/m/011_vz': 31,
             '/m/011hdn': 32,
      